### Model notes:
- Only 60 mo product
- Use SMOTE to upsample minority class and gridsearch

#### Results: much worse than the 36mo product
loan approval rate: 0.53\
precision pred_1: 0.82\
recall pred_1: 0.57

In [165]:
#Installing imblearn
#!pip install -U imbalanced-learn

In [166]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 50)

In [167]:
%%time
df = pd.read_parquet('../data/approved.parquet', engine='fastparquet')

Wall time: 2.91 s


In [168]:
df.sample(5)

,addr_state,annual_inc,application_type,disbursement_method,dti,earliest_cr_line,emp_length,emp_title,fico_range_high,fico_range_low,grade,home_ownership,initial_list_status,installment,int_rate,issue_d,loan_amnt,open_acc,pub_rec,pub_rec_bankruptcies,purpose,sub_grade,term,verification_status,zip_code,loan_status
id,,,,,,,,,,,,,,,,,,,,,,,,,,
23885541,NJ,76290.0,Individual,Cash,19.84,1998-01-01,10,Operations Manager,734.0,730.0,C,MORTGAGE,f,673.79,12.99,2014-10-01,20000.0,9.0,0.0,0.0,credit_card,C1,36,Source Verified,081xx,0
108719265,MI,100000.0,Individual,Cash,6.60,2004-09-01,5,contracts manager,774.0,770.0,A,RENT,w,602.30,5.32,2017-06-01,20000.0,8.0,0.0,0.0,debt_consolidation,A1,36,Not Verified,481xx,1
111999165,CA,180000.0,Individual,Cash,17.73,2006-09-01,9,Project Manager,669.0,665.0,D,RENT,f,695.89,20.00,2017-07-01,18725.0,8.0,0.0,0.0,credit_card,D4,36,Verified,941xx,0
119281514,WA,96000.0,Joint App,Cash,15.67,1991-08-01,1,Director of Capital Development,744.0,740.0,B,MORTGAGE,w,461.40,9.44,2017-09-01,22000.0,14.0,0.0,0.0,medical,B1,60,Source Verified,983xx,1
11375329,NY,105000.0,Individual,Cash,23.77,1995-08-01,1,Business Development Strategist,679.0,675.0,B,RENT,f,353.24,9.67,2014-01-01,11000.0,8.0,0.0,0.0,credit_card,B1,36,Source Verified,100xx,1


### Feature Engineering

In [169]:
df['days_cr_line']=df['issue_d']-df['earliest_cr_line']
#convert to int (https://www.codegrepper.com/code-examples/python/pandas+timedelta64+ns+to+float)
df['days_cr_line'] = (df['days_cr_line'] / np.timedelta64(1,'D')).astype(int)

### Convert Ordinals

In [170]:
# sub_grade
df.sub_grade.replace({'A1':1, 'A2':2, 'A3':3, 'A4':4, 'A5':5, 'B1':6, 'B2':7, 
                      'B3':8, 'B4':9, 'B5':10, 'C1':11, 'C2':12, 'C3':13, 'C4':14, 
                      'C5':15, 'D1':16, 'D2':17, 'D3':18, 'D4':19, 'D5':20, 'E1':21, 
                      'E2':22, 'E3':23, 'E4':24, 'E5':25}, inplace=True)

# disbursement_method
df.disbursement_method.replace({'Cash':0,'DirectPay':1},inplace=True)

# verification_status
df.verification_status.replace({'Source Verified':'Verified'},inplace=True)
df.verification_status.replace({'Not Verified':0,'Verified':1}, inplace=True)

# initial_list_status
df.initial_list_status.replace({'w':0,'f':1}, inplace=True)

# application_type
df.application_type.replace({'Individual':0,'Joint App':1}, inplace=True)

### Dummification

In [171]:
dummies=['purpose','addr_state','home_ownership']

for d in dummies:
    temp = pd.get_dummies(df[d], prefix=d, prefix_sep='__')
    temp = temp.drop(d+'__'+str(df[d].mode()[0]), axis=1)
    #drop the most common one (mode), making it the base case
    df = pd.concat([df.drop(d, axis=1),temp], axis=1)
    #merge back with main df

### Filter for years < 2013 because this is just the 60mo product

In [172]:
df = df.loc[df['issue_d'].dt.year.between(2007,2013)]

### Drop Columns

In [173]:
df.drop(['emp_title','zip_code','grade','issue_d','earliest_cr_line'],axis=1, inplace=True)
#removed 'issue_d'

### Drop Rows

In [174]:
df.drop(df[df['emp_length']==-1].index, inplace=True)

### Drop the 60month product

In [175]:
df = df.loc[df['term']==60]
df.drop('term', axis=1, inplace=True)

# Modelling

### create target and features

In [176]:
target = df['loan_status']

In [177]:
target.value_counts()

1    35614
0    10901
Name: loan_status, dtype: int64

In [178]:
features = df.drop(['loan_status'], axis=1)

### note the reduced train_size of .10 = ~120,000 obs

In [180]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, KFold

Xtrain, Xtest, ytrain, ytest = train_test_split(features, target, train_size=.70 )
logr = LogisticRegression()

In [181]:
# logr.get_params()

### SMOTE to oversample minority class

In [182]:
%%time
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 101)
X_oversample, y_oversample = smote.fit_resample(Xtrain, ytrain)

Wall time: 1.98 s


In [183]:
print(y_oversample.value_counts())
print(ytest.value_counts())

1    24910
0    24910
Name: loan_status, dtype: int64
1    10704
0     3251
Name: loan_status, dtype: int64


In [184]:
X_oversample, y_oversample = smote.fit_resample(Xtrain, ytrain)

### gridsearch

In [185]:
Cs = [0.0001,0.001,0.01,1,10,100,1000,10000]
params = [{'C':Cs, 'penalty':['l1','l2']}]
grid_logr = GridSearchCV(estimator=logr, param_grid=params, cv=3)

In [212]:
%%time
grid_logr.fit(X_oversample, y_oversample)

Wall time: 10.3 s


GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid=[{'C': [0.0001, 0.001, 0.01, 1, 10, 100, 1000, 10000],
                          'penalty': ['l1', 'l2']}])

In [213]:
print(grid_logr.best_params_)
print(grid_logr.best_score_)

{'C': 10, 'penalty': 'l2'}
0.5849056991245326


In [214]:
best_model = grid_logr.best_estimator_
best_model.score(Xtest,ytest)

0.5643138659978503

In [215]:
coefs = pd.DataFrame(abs(best_model.coef_).T, index=features.columns)

In [216]:
coefs.sort_values(0, ascending=False).head(10)

,0
installment,0.006901
sub_grade,0.000891
int_rate,0.000756
dti,0.000545
fico_range_high,0.000229
open_acc,0.000220
fico_range_low,0.000202
loan_amnt,0.000162
emp_length,0.000111
purpose__credit_card,0.000059


### Confusion Matrix

In [217]:
from sklearn.metrics import confusion_matrix

In [218]:
cmat = confusion_matrix(ytest,best_model.predict(Xtest))

#Label the confusion matrix
pd.DataFrame(cmat, columns = [f'Pred_{label}' for label in best_model.classes_],
             index = [f'True_{label}' for label in best_model.classes_])


,Pred_0,Pred_1
True_0,1984,1267
True_1,4813,5891


### Best model is one that strikes good balance of a high "pred_1 precision" and a high "loan approval rate"

In [219]:
cmat=cmat
print(f'loan approval rate: {sum(cmat[:,1])/sum(sum(cmat)):.2f}')
print(f'precision pred_1: {cmat[1][1]/sum(cmat[:,1]):.2f}')
print(f'recall pred_1: {cmat[1][1]/sum(cmat[1,:]):.2f}')

loan approval rate: 0.51
precision pred_1: 0.82
recall pred_1: 0.55


In [220]:
from sklearn.metrics import classification_report
print(classification_report(ytest, best_model.predict(Xtest)))

              precision    recall  f1-score   support

           0       0.29      0.61      0.39      3251
           1       0.82      0.55      0.66     10704

    accuracy                           0.56     13955
   macro avg       0.56      0.58      0.53     13955
weighted avg       0.70      0.56      0.60     13955



### change the threshold to improve the pred_1 precision

In [221]:
probs_positive_class = best_model.predict_proba(Xtest)[:, 1]
prediction = probs_positive_class > .6

In [222]:
cmat2 = confusion_matrix(ytest, prediction, labels=[0, 1])
pd.DataFrame(cmat2, columns = [f'Pred_{label}' for label in best_model.classes_],
             index = [f'True_{label}' for label in best_model.classes_])

,Pred_0,Pred_1
True_0,2944,307
True_1,9069,1635


In [223]:
from sklearn.metrics import classification_report
print(classification_report(ytest, prediction))

              precision    recall  f1-score   support

           0       0.25      0.91      0.39      3251
           1       0.84      0.15      0.26     10704

    accuracy                           0.33     13955
   macro avg       0.54      0.53      0.32     13955
weighted avg       0.70      0.33      0.29     13955



In [224]:
cmat=cmat2
print(f'loan approval rate: {sum(cmat[:,1])/sum(sum(cmat)):.2f}')
print(f'precision pred_1: {cmat[1][1]/sum(cmat[:,1]):.2f}')
print(f'recall pred_1: {cmat[1][1]/sum(cmat[1,:]):.2f}')

loan approval rate: 0.14
precision pred_1: 0.84
recall pred_1: 0.15


### Loop to check thresholds

In [241]:
thresholds = np.linspace(0.5,1,19)

for i in thresholds:
    probs_positive_class = best_model.predict_proba(Xtest)[:, 1]
    prediction = probs_positive_class > i
    cmat = confusion_matrix(ytest, prediction, labels=[0, 1])
    print(f'threshold:{i:.2f}')
    print(f'loan approval rate: {sum(cmat[:,1])/sum(sum(cmat)):.2f}')
    print(f'precision pred_1: {cmat[1][1]/sum(cmat[:,1]):.2f}')
    print(f'recall pred_1: {cmat[1][1]/sum(cmat[1,:]):.2f}')
    print("-"*50)

threshold:0.50
loan approval rate: 0.51
precision pred_1: 0.82
recall pred_1: 0.55
--------------------------------------------------
threshold:0.53
loan approval rate: 0.37
precision pred_1: 0.84
recall pred_1: 0.41
--------------------------------------------------
threshold:0.56
loan approval rate: 0.26
precision pred_1: 0.85
recall pred_1: 0.29
--------------------------------------------------
threshold:0.58
loan approval rate: 0.18
precision pred_1: 0.85
recall pred_1: 0.20
--------------------------------------------------
threshold:0.61
loan approval rate: 0.12
precision pred_1: 0.84
recall pred_1: 0.13
--------------------------------------------------
threshold:0.64
loan approval rate: 0.08
precision pred_1: 0.84
recall pred_1: 0.09
--------------------------------------------------
threshold:0.67
loan approval rate: 0.05
precision pred_1: 0.83
recall pred_1: 0.06
--------------------------------------------------
threshold:0.69
loan approval rate: 0.04
precision pred_1: 0.82

## ROC Curve

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
# predict probabilities
yhat = logr.predict_proba(Xtest)

In [ ]:
# keep probabilities for the positive outcome only
yhat = yhat[:, 1]

In [ ]:
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

In [ ]:
# plot the roc curve for the model
plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label='Logistic')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
plt.show()

In [ ]:
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, prediction)

In [ ]:
# plot the roc curve for the model
plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label='Logistic')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
plt.show()

### SGD classifier

### Save model in pickle

In [ ]:
filename = 'grid_logr_model.sav'
pickle.dump(grid_logr, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(Xtest, ytest)
print(result)